In [1]:
import os
import sys
import json
import torch
import importlib
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn as nn
import pandas as pd
from typing import Dict, List, Any
from torch.utils.data import DataLoader, TensorDataset

# Get the current working directory
scripts_dir = os.getcwd()
# Go up one level
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)

import src.utils
importlib.reload(src.utils)
from src.utils import calculate_sum_rate_sc

import src.sc_wmmse
importlib.reload(src.sc_wmmse)
from src.sc_wmmse import WMMSE_alg_sc

In [ ]:
# Implement the deep-unfolding NN architecture
import torch
import torch.nn as nn
import pandas as pd
from typing import Dict, List, Any
from torch.utils.data import DataLoader, TensorDataset

# Define the layer
class Layer(nn.Module):
    def __init__(self, setup, flagg):
        super().__init__()
        self.setup = setup
        X_U_dict = {}
        Y_U_dict = {}
        Z_U_dict = {}
        O_U_dict = {}
        X_W_dict = {}
        Y_W_dict = {}
        Z_W_dict = {}
        X_V_dict = {}
        Y_V_dict = {}
        Z_V_dict = {}
        O_V_dict = {}
        for i in range(self.setup.K):
            # X_U_dict[str(i)] = nn.Parameter(m01)
            # Y_U_dict[str(i)] = nn.Parameter(m02)
            # Z_U_dict[str(i)] = nn.Parameter(m03)
            # O_U_dict[str(i)] = nn.Parameter(m04)
            # X_W_dict[str(i)] = nn.Parameter(m05)
            # Y_W_dict[str(i)] = nn.Parameter(m06)
            # Z_W_dict[str(i)] = nn.Parameter(m07)
            # X_V_dict[str(i)] = nn.Parameter(m08)
            # Y_V_dict[str(i)] = nn.Parameter(m09)
            # Z_V_dict[str(i)] = nn.Parameter(m010)
            # O_V_dict[str(i)] = nn.Parameter(m011)
            X_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
            Y_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
            Z_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
            O_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.d[i], dtype=torch.cdouble))
            X_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
            Y_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
            Z_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
            X_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
            Y_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
            Z_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
            O_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.d[i], dtype=torch.cdouble))
            # X_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
            # Y_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
            # Z_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
            # O_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.d[i], dtype=torch.cdouble))
            # X_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
            # Y_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
            # Z_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
            # X_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
            # Y_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
            # Z_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
            # O_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.d[i], dtype=torch.cdouble))
        self.X_U = nn.ParameterDict(X_U_dict)
        self.Y_U = nn.ParameterDict(Y_U_dict)
        self.Z_U = nn.ParameterDict(Z_U_dict)
        self.O_U = nn.ParameterDict(O_U_dict)
        self.X_W = nn.ParameterDict(X_W_dict)
        self.Y_W = nn.ParameterDict(Y_W_dict)
        self.Z_W = nn.ParameterDict(Z_W_dict)
        self.X_V = nn.ParameterDict(X_V_dict)
        self.Y_V = nn.ParameterDict(Y_V_dict)
        self.Z_V = nn.ParameterDict(Z_V_dict)
        self.O_V = nn.ParameterDict(O_V_dict)


    def forward(self, V, H):

        # A^+ operator
        # def plus(A):
        #     diag_inv = 1.0 / A.diagonal(dim1=-2, dim2=-1)
        #     A_plus = torch.zeros_like(A)
        #     A_plus.diagonal(dim1=-2, dim2=-1).copy_(diag_inv)
        #     return A_plus
        # def plus(A: torch.Tensor) -> torch.Tensor:
        #     """
        #     Moore-Penrose pseudo-inverse of a diagonal matrix: keeps autograd path.
        #     Works batch-wise for ...×N×N tensors.
        #     """
        #     diag = torch.diagonal(A, 0, -2, -1)          # keeps grad wrt A
        #     inv  = 1.0 / diag                            # element-wise inverse
        #     return torch.diag_embed(inv)
        def plus(A):
            return A

        def proj_power(V):
            V_proj = {}
            for i in range(num_samples):
                V_proj[str(i)] = {}
                s = 0
                for j in range(self.setup.K):
                    s += torch.trace(V[str(i)][str(j)] @ V[str(i)][str(j)].conj().T)
                for j in range(self.setup.K):
                    V_proj[str(i)][str(j)] = torch.sqrt(self.setup.PT/s) * V[str(i)][str(j)]
            return V_proj

        # def proj_power(V_in: dict) -> dict:
        #     """
        #     Return a *new* dictionary whose entries are power-normalised copies
        #     of the Tx precoders in V_in.  No in-place writes → autograd path stays
        #     intact when multiple unfolded layers are stacked.

        #     V_in structure (unchanged):  {sample: {user: V_k}}
        #     """
        #     V_out = {}
        #     for sample, users in V_in.items():

        #         # total power  (trace is complex → take real part)
        #         P_tot = sum(torch.trace(v @ v.conj().T).real for v in users.values())
        #         scale = torch.sqrt(self.setup.PT / (P_tot + 1e-12))     # +ε avoids div-by-0

        #         V_out[sample] = {user: scale * v             # *clone* not needed
        #                         for user, v in users.items()}
        #     return V_out

        # def proj_power_soft(V_in: dict, PT: float, alpha: float = 0.1) -> dict:
        #     V_out = {}
        #     for i, users in V_in.items():
        #         P_tot = sum(torch.trace(v @ v.conj().T).real for v in users.values())
        #         scale = torch.sqrt(PT / (P_tot + 1e-12))
        #         V_out[str(i)] = {
        #             str(k): (1 - alpha) * v + alpha * scale * v for k, v in users.items()
        #         }
        #     return V_out


        num_samples = len(H)

        # Calculate A
        A = {}
        for i in range(num_samples):
            A[str(i)] = {}
            for j in range(self.setup.K):
                s = 0
                for k in range(self.setup.K):
                    s += torch.trace(V[str(i)][str(k)].conj().T @ V[str(i)][str(k)])
                ey = (1/self.setup.PT) * s * torch.eye(self.setup.n_rx[j], dtype=torch.cdouble)
                s = 0
                for k in range(self.setup.K):
                    s += V[str(i)][str(k)] @ V[str(i)][str(k)].conj().T
                A[str(i)][str(j)] = ey + H.iloc[i, j] @ s @ H.iloc[i, j].conj().T

        # Calculate U
        U = {}
        for i in range(num_samples):
            U[str(i)] = {}
            for j in range(self.setup.K):
                A_inv = plus(A[str(i)][str(j)]) @ self.X_U[str(j)].to(torch.cdouble) + A[str(i)][str(j)] @ self.Y_U[str(j)].to(torch.cdouble) + self.Z_U[str(j)].to(torch.cdouble)
                U[str(i)][str(j)] = A_inv @ H.iloc[i, j] @ V[str(i)][str(j)] + self.O_U[str(j)].to(torch.cdouble)

        # Calclate E
        E = {}
        for i in range(num_samples):
            E[str(i)] = {}
            for j in range(self.setup.K):
                E[str(i)][str(j)] = torch.eye(self.setup.d[j], dtype=torch.cdouble) - U[str(i)][str(j)].conj().T @ H.iloc[i, j] @ V[str(i)][str(j)]
        
        # Calculate W
        W = {}
        for i in range(num_samples):
            W[str(i)] = {}
            for j in range(self.setup.K):
                W[str(i)][str(j)] = plus(E[str(i)][str(j)]) @ self.X_W[str(j)].to(torch.cdouble) + E[str(i)][str(j)] @ self.Y_W[str(j)].to(torch.cdouble) + self.Z_W[str(j)].to(torch.cdouble)

        # Calculate B
        B = {}
        for i in range(num_samples):
            s = 0
            for k in range(self.setup.K):
                s += torch.trace(U[str(i)][str(k)] @ W[str(i)][str(k)] @ U[str(i)][str(k)].conj().T)
            ey = (1/self.setup.PT) * s * torch.eye(self.setup.n_tx, dtype=torch.cdouble)
            s = 0
            for k in range(self.setup.K):
                s += H.iloc[i, k].conj().T @ U[str(i)][str(k)] @ W[str(i)][str(k)] @ U[str(i)][str(k)].conj().T @ H.iloc[i, k]
            B[str(i)] = ey + s
                

        # Calculate V
        V = {}
        for i in range(num_samples):
            V[str(i)] = {}
            for j in range(self.setup.K): 
                B_inv = plus(B[str(i)]) @ self.X_V[str(j)].to(torch.cdouble) + B[str(i)] @ self.Y_V[str(j)].to(torch.cdouble) + self.Z_V[str(j)].to(torch.cdouble)
                V[str(i)][str(j)] = B_inv @ H.iloc[i, j].conj().T @ U[str(i)][str(j)] @ W[str(i)][str(j)] + self.O_V[str(j)].to(torch.cdouble)

        # Project V
        V_proj = proj_power(V)
        # V_proj = proj_power_soft(V, self.setup.PT, alpha=0.1)

        return V_proj

# Define the deep unfolding NN
class DUNN(nn.Module):
    def __init__(self, num_layers, setup):
        super().__init__()
        self.layers = nn.ModuleList([
            Layer(setup, i)
            for i in range(num_layers)
        ])

    def forward(self, V, H):
        for layer in self.layers:
            V = layer(V, H)
        return V

# Train the model
class Trainer:
    def __init__(self, model: DUNN, setup, lr: float = 1e-3):
        self.setup = setup
        self.model = model
        self.opt = torch.optim.Adam(model.parameters(), lr=lr)

    def train_epoch(self, dset, loss_fn, num_epochs, batch_size):

        # Function for initializing V
        # def init_V(H_df, setup):
        #     num_samples = H_df.shape[0]
        #     K = setup.K
        #     V_dict = {}

        #     for sample_idx in range(num_samples):
        #         V_sample = {}
        #         H_sample = [H_df.iloc[sample_idx, k] for k in range(K)]

        #         for k in range(K):
        #             # Create interference channel matrix for all users ≠ k
        #             H_interference = torch.cat([H_sample[j] for j in range(K) if j != k], dim=0)

        #             # Compute null space of interference channel using SVD
        #             _, S, Vh = torch.linalg.svd(H_interference)
        #             rank = (S > 1e-6).sum().item()
        #             null_space = Vh[rank:].conj().T  # shape: [n_tx, nullity]

        #             # Choose as many columns as the number of streams we want (≤ nullity)
        #             d_k = min(setup.n_rx[k], null_space.shape[1])
        #             V_k = null_space[:, :d_k]

        #             V_sample[str(k)] = V_k

        #         V_dict[str(sample_idx)] = V_sample
        #     return V_dict

        def proj_power(V):
            for i in range(len(V)):
                s = 0
                for j in range(self.setup.K):
                    s += torch.trace(V[str(i)][str(j)] @ V[str(i)][str(j)].conj().T)
                for j in range(self.setup.K):
                    V[str(i)][str(j)] = torch.sqrt(self.setup.PT/s) * V[str(i)][str(j)]
            return V

        def shuffle_and_batch(df, batch_size):
            df_shuffled = df.sample(frac=1).reset_index(drop=True)
            return [df_shuffled[i:i + batch_size] for i in range(0, len(df_shuffled), batch_size)]

        loader = shuffle_and_batch(dset, batch_size)

        H_total = dset.iloc[:, :self.setup.K]
        V_total_df = dset.iloc[:, self.setup.K:(2*self.setup.K)]
        V_total = {
                    str(i): {str(j): V_total_df.iloc[i, j] for j in range(V_total_df.shape[1])}
                    for i in range(len(V_total_df))
                }
        r_l = []
        for _ in range(num_epochs):
            self.model.train()
            total_loss = 0.0
            for dset_batch in loader:
                H_batch = dset_batch.iloc[:, :self.setup.K]
                V_init_batch = dset_batch.iloc[:, self.setup.K:(2*self.setup.K)]
                V0 = {
                    str(i): {str(j): V_init_batch.iloc[i, j] for j in range(V_init_batch.shape[1])}
                    for i in range(len(V_init_batch))
                }
                # H_batch = H_batch.to(torch.complex64)
                self.opt.zero_grad()
                # Initialize V
                # V0 = init_V(H_batch, self.setup)
                # V0 = proj_power(V0)
                # V0 = V_init
                V_pred = self.model(V0, H_batch)
                loss = (-1) * loss_fn(H_batch, V_pred, self.setup.PT)
                print(loss)
                loss.backward()
                # print(self.model.layers[0].X_U['0'].grad)
                self.opt.step()
                total_loss += loss.item()
            # r_l.append(loss_fn(H_total, self.model(V_total, H_total), self.setup.PT))

        # return total_loss / len(loader)
        # return r_l
        return V_pred

In [ ]:
# Implement the deep-unfolding NN architecture
import torch
import torch.nn as nn
import pandas as pd
from typing import Dict, List, Any
from torch.utils.data import DataLoader, TensorDataset

# Define the layer
class Layer(nn.Module):
    def __init__(self, setup, flagg):
        super().__init__()
        self.setup = setup
        X_U_dict = {}
        Y_U_dict = {}
        Z_U_dict = {}
        O_U_dict = {}
        X_W_dict = {}
        Y_W_dict = {}
        Z_W_dict = {}
        X_V_dict = {}
        Y_V_dict = {}
        Z_V_dict = {}
        O_V_dict = {}

        if flagg == 0:
            print('h')
            for i in range(self.setup.K):
                X_U_dict[str(i)] = nn.Parameter(m1)
                Y_U_dict[str(i)] = nn.Parameter(m2)
                Z_U_dict[str(i)] = nn.Parameter(m3)
                O_U_dict[str(i)] = nn.Parameter(m4)
                X_W_dict[str(i)] = nn.Parameter(m5)
                Y_W_dict[str(i)] = nn.Parameter(m6)
                Z_W_dict[str(i)] = nn.Parameter(m7)
                X_V_dict[str(i)] = nn.Parameter(m8)
                Y_V_dict[str(i)] = nn.Parameter(m9)
                Z_V_dict[str(i)] = nn.Parameter(m10)
                O_V_dict[str(i)] = nn.Parameter(m11)
                # X_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # Y_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # Z_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # O_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.d[i], dtype=torch.cdouble))
                # X_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # Y_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # Z_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # X_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # Y_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # Z_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # O_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.d[i], dtype=torch.cdouble))
                # # X_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # # Y_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # # Z_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # # O_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.d[i], dtype=torch.cdouble))
                # # X_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # # Y_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # # Z_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # # X_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # # Y_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # # Z_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # # O_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.d[i], dtype=torch.cdouble))
            self.X_U = nn.ParameterDict(X_U_dict)
            self.Y_U = nn.ParameterDict(Y_U_dict)
            self.Z_U = nn.ParameterDict(Z_U_dict)
            self.O_U = nn.ParameterDict(O_U_dict)
            self.X_W = nn.ParameterDict(X_W_dict)
            self.Y_W = nn.ParameterDict(Y_W_dict)
            self.Z_W = nn.ParameterDict(Z_W_dict)
            self.X_V = nn.ParameterDict(X_V_dict)
            self.Y_V = nn.ParameterDict(Y_V_dict)
            self.Z_V = nn.ParameterDict(Z_V_dict)
            self.O_V = nn.ParameterDict(O_V_dict)

        if flagg == 1:
            print('h')
            for i in range(self.setup.K):
                X_U_dict[str(i)] = nn.Parameter(m01)
                Y_U_dict[str(i)] = nn.Parameter(m02)
                Z_U_dict[str(i)] = nn.Parameter(m03)
                O_U_dict[str(i)] = nn.Parameter(m04)
                X_W_dict[str(i)] = nn.Parameter(m05)
                Y_W_dict[str(i)] = nn.Parameter(m06)
                Z_W_dict[str(i)] = nn.Parameter(m07)
                X_V_dict[str(i)] = nn.Parameter(m08)
                Y_V_dict[str(i)] = nn.Parameter(m09)
                Z_V_dict[str(i)] = nn.Parameter(m010)
                O_V_dict[str(i)] = nn.Parameter(m011)
                # X_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # Y_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # Z_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # O_U_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_rx[i], self.setup.d[i], dtype=torch.cdouble))
                # X_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # Y_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # Z_W_dict[str(i)] = nn.Parameter(torch.randn(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # X_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # Y_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # Z_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # O_V_dict[str(i)] = nn.Parameter(torch.randn(self.setup.n_tx, self.setup.d[i], dtype=torch.cdouble))
                # # X_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # # Y_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # # Z_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.n_rx[i], dtype=torch.cdouble))
                # # O_U_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_rx[i], self.setup.d[i], dtype=torch.cdouble))
                # # X_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # # Y_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # # Z_W_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.d[i], self.setup.d[i], dtype=torch.cdouble))
                # # X_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # # Y_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # # Z_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.n_tx, dtype=torch.cdouble))
                # # O_V_dict[str(i)] = nn.Parameter(100 * torch.rand(self.setup.n_tx, self.setup.d[i], dtype=torch.cdouble))
            self.X_U = nn.ParameterDict(X_U_dict)
            self.Y_U = nn.ParameterDict(Y_U_dict)
            self.Z_U = nn.ParameterDict(Z_U_dict)
            self.O_U = nn.ParameterDict(O_U_dict)
            self.X_W = nn.ParameterDict(X_W_dict)
            self.Y_W = nn.ParameterDict(Y_W_dict)
            self.Z_W = nn.ParameterDict(Z_W_dict)
            self.X_V = nn.ParameterDict(X_V_dict)
            self.Y_V = nn.ParameterDict(Y_V_dict)
            self.Z_V = nn.ParameterDict(Z_V_dict)
            self.O_V = nn.ParameterDict(O_V_dict)


    def forward(self, V, H):

        # A^+ operator
        # def plus(A):
        #     diag_inv = 1.0 / A.diagonal(dim1=-2, dim2=-1)
        #     A_plus = torch.zeros_like(A)
        #     A_plus.diagonal(dim1=-2, dim2=-1).copy_(diag_inv)
        #     return A_plus
        # def plus(A: torch.Tensor) -> torch.Tensor:
        #     """
        #     Moore-Penrose pseudo-inverse of a diagonal matrix: keeps autograd path.
        #     Works batch-wise for ...×N×N tensors.
        #     """
        #     diag = torch.diagonal(A, 0, -2, -1)          # keeps grad wrt A
        #     inv  = 1.0 / diag                            # element-wise inverse
        #     return torch.diag_embed(inv)
        def plus(A):
            return A

        def proj_power(V):
            V_proj = {}
            for i in range(num_samples):
                V_proj[str(i)] = {}
                s = 0
                for j in range(self.setup.K):
                    s += torch.trace(V[str(i)][str(j)] @ V[str(i)][str(j)].conj().T)
                for j in range(self.setup.K):
                    V_proj[str(i)][str(j)] = torch.sqrt(self.setup.PT/s) * V[str(i)][str(j)]
            return V_proj

        # def proj_power(V_in: dict) -> dict:
        #     """
        #     Return a *new* dictionary whose entries are power-normalised copies
        #     of the Tx precoders in V_in.  No in-place writes → autograd path stays
        #     intact when multiple unfolded layers are stacked.

        #     V_in structure (unchanged):  {sample: {user: V_k}}
        #     """
        #     V_out = {}
        #     for sample, users in V_in.items():

        #         # total power  (trace is complex → take real part)
        #         P_tot = sum(torch.trace(v @ v.conj().T).real for v in users.values())
        #         scale = torch.sqrt(self.setup.PT / (P_tot + 1e-12))     # +ε avoids div-by-0

        #         V_out[sample] = {user: scale * v             # *clone* not needed
        #                         for user, v in users.items()}
        #     return V_out

        # def proj_power_soft(V_in: dict, PT: float, alpha: float = 0.1) -> dict:
        #     V_out = {}
        #     for i, users in V_in.items():
        #         P_tot = sum(torch.trace(v @ v.conj().T).real for v in users.values())
        #         scale = torch.sqrt(PT / (P_tot + 1e-12))
        #         V_out[str(i)] = {
        #             str(k): (1 - alpha) * v + alpha * scale * v for k, v in users.items()
        #         }
        #     return V_out


        num_samples = len(H)

        # Calculate A
        A = {}
        for i in range(num_samples):
            A[str(i)] = {}
            for j in range(self.setup.K):
                s = 0
                for k in range(self.setup.K):
                    s += torch.trace(V[str(i)][str(k)].conj().T @ V[str(i)][str(k)])
                ey = (1/self.setup.PT) * s * torch.eye(self.setup.n_rx[j], dtype=torch.cdouble)
                s = 0
                for k in range(self.setup.K):
                    s += V[str(i)][str(k)] @ V[str(i)][str(k)].conj().T
                A[str(i)][str(j)] = ey + H.iloc[i, j] @ s @ H.iloc[i, j].conj().T

        # Calculate U
        U = {}
        for i in range(num_samples):
            U[str(i)] = {}
            for j in range(self.setup.K):
                A_inv = plus(A[str(i)][str(j)]) @ self.X_U[str(j)].to(torch.cdouble) + A[str(i)][str(j)] @ self.Y_U[str(j)].to(torch.cdouble) + self.Z_U[str(j)].to(torch.cdouble)
                U[str(i)][str(j)] = A_inv @ H.iloc[i, j] @ V[str(i)][str(j)] + self.O_U[str(j)].to(torch.cdouble)

        # Calclate E
        E = {}
        for i in range(num_samples):
            E[str(i)] = {}
            for j in range(self.setup.K):
                E[str(i)][str(j)] = torch.eye(self.setup.d[j], dtype=torch.cdouble) - U[str(i)][str(j)].conj().T @ H.iloc[i, j] @ V[str(i)][str(j)]
        
        # Calculate W
        W = {}
        for i in range(num_samples):
            W[str(i)] = {}
            for j in range(self.setup.K):
                W[str(i)][str(j)] = plus(E[str(i)][str(j)]) @ self.X_W[str(j)].to(torch.cdouble) + E[str(i)][str(j)] @ self.Y_W[str(j)].to(torch.cdouble) + self.Z_W[str(j)].to(torch.cdouble)

        # Calculate B
        B = {}
        for i in range(num_samples):
            s = 0
            for k in range(self.setup.K):
                s += torch.trace(U[str(i)][str(k)] @ W[str(i)][str(k)] @ U[str(i)][str(k)].conj().T)
            ey = (1/self.setup.PT) * s * torch.eye(self.setup.n_tx, dtype=torch.cdouble)
            s = 0
            for k in range(self.setup.K):
                s += H.iloc[i, k].conj().T @ U[str(i)][str(k)] @ W[str(i)][str(k)] @ U[str(i)][str(k)].conj().T @ H.iloc[i, k]
            B[str(i)] = ey + s
                

        # Calculate V
        V = {}
        for i in range(num_samples):
            V[str(i)] = {}
            for j in range(self.setup.K): 
                B_inv = plus(B[str(i)]) @ self.X_V[str(j)].to(torch.cdouble) + B[str(i)] @ self.Y_V[str(j)].to(torch.cdouble) + self.Z_V[str(j)].to(torch.cdouble)
                V[str(i)][str(j)] = B_inv @ H.iloc[i, j].conj().T @ U[str(i)][str(j)] @ W[str(i)][str(j)] + self.O_V[str(j)].to(torch.cdouble)

        # Project V
        V_proj = proj_power(V)
        # V_proj = proj_power_soft(V, self.setup.PT, alpha=0.1)

        return V_proj

# Define the deep unfolding NN
class DUNN(nn.Module):
    def __init__(self, num_layers, setup):
        super().__init__()
        self.layers = nn.ModuleList([
            Layer(setup, i)
            for i in range(num_layers)
        ])

    def forward(self, V, H):
        for layer in self.layers:
            V = layer(V, H)
        return V

# Train the model
class Trainer:
    def __init__(self, model: DUNN, setup, lr: float = 1e-3):
        self.setup = setup
        self.model = model
        self.opt = torch.optim.Adam(model.parameters(), lr=lr)

    def train_epoch(self, dset, loss_fn, num_epochs, batch_size):

        # Function for initializing V
        # def init_V(H_df, setup):
        #     num_samples = H_df.shape[0]
        #     K = setup.K
        #     V_dict = {}

        #     for sample_idx in range(num_samples):
        #         V_sample = {}
        #         H_sample = [H_df.iloc[sample_idx, k] for k in range(K)]

        #         for k in range(K):
        #             # Create interference channel matrix for all users ≠ k
        #             H_interference = torch.cat([H_sample[j] for j in range(K) if j != k], dim=0)

        #             # Compute null space of interference channel using SVD
        #             _, S, Vh = torch.linalg.svd(H_interference)
        #             rank = (S > 1e-6).sum().item()
        #             null_space = Vh[rank:].conj().T  # shape: [n_tx, nullity]

        #             # Choose as many columns as the number of streams we want (≤ nullity)
        #             d_k = min(setup.n_rx[k], null_space.shape[1])
        #             V_k = null_space[:, :d_k]

        #             V_sample[str(k)] = V_k

        #         V_dict[str(sample_idx)] = V_sample
        #     return V_dict

        def proj_power(V):
            for i in range(len(V)):
                s = 0
                for j in range(self.setup.K):
                    s += torch.trace(V[str(i)][str(j)] @ V[str(i)][str(j)].conj().T)
                for j in range(self.setup.K):
                    V[str(i)][str(j)] = torch.sqrt(self.setup.PT/s) * V[str(i)][str(j)]
            return V

        def shuffle_and_batch(df, batch_size):
            df_shuffled = df.sample(frac=1).reset_index(drop=True)
            return [df_shuffled[i:i + batch_size] for i in range(0, len(df_shuffled), batch_size)]

        loader = shuffle_and_batch(dset, batch_size)

        H_total = dset.iloc[:, :self.setup.K]
        V_total_df = dset.iloc[:, self.setup.K:(2*self.setup.K)]
        V_total = {
                    str(i): {str(j): V_total_df.iloc[i, j] for j in range(V_total_df.shape[1])}
                    for i in range(len(V_total_df))
                }
        r_l = []
        for _ in range(num_epochs):
            self.model.train()
            total_loss = 0.0
            for dset_batch in loader:
                H_batch = dset_batch.iloc[:, :self.setup.K]
                V_init_batch = dset_batch.iloc[:, self.setup.K:(2*self.setup.K)]
                V0 = {
                    str(i): {str(j): V_init_batch.iloc[i, j] for j in range(V_init_batch.shape[1])}
                    for i in range(len(V_init_batch))
                }
                # H_batch = H_batch.to(torch.complex64)
                self.opt.zero_grad()
                # Initialize V
                # V0 = init_V(H_batch, self.setup)
                # V0 = proj_power(V0)
                # V0 = V_init
                V_pred = self.model(V0, H_batch)
                loss = (-1) * loss_fn(H_batch, V_pred, self.setup.PT)
                print(loss)
                loss.backward()
                # print(self.model.layers[0].X_U['0'].grad)
                self.opt.step()
                total_loss += loss.item()
            # r_l.append(loss_fn(H_total, self.model(V_total, H_total), self.setup.PT))

        # return total_loss / len(loader)
        # return r_l
        return V_pred

In [2]:
def proj_power(V, PT_sc):
    num_users_sc = len(V)
    # Projects V according to the constraint
    alph = torch.sqrt(torch.tensor(PT_sc)) / torch.sqrt(torch.tensor(sum([torch.trace(V[str(k)] @ V[str(k)].conj().T) for k in range(num_users_sc)])))
    V_proj = {str(k): alph * V[str(k)] for k in range(num_users_sc)}
    return V_proj

def init_V(H):
    # Initializes V according to Hu's code
    V = {}
    for k in range(len(H_dict)):
        V[str(k)] = (torch.linalg.pinv(H[str(k)] @ H[str(k)].conj().T) @ H[str(k)]).conj().T
    return V

In [ ]:
class setup():
    def __init__(self, n_tx, n_rx, num_streams, num_users, PT, sig):
        self.n_tx = n_tx
        self.n_rx = n_rx
        self.d = num_streams
        self.K = num_users
        self.PT = PT
        self.sig = sig

In [ ]:
# The setup
num_users = 1
n_tx = 4
n_rx = [2] * num_users
d = [2] * num_users
PT = 200
sig = [1] * num_users
alpha = [1] * num_users
max_iter_alg = 100
tol_alg = 1e-3

In [ ]:
data = []
for _ in range(1):  # 5 rows
    row = {f'user_{i}': torch.randn(n_rx[i], n_tx, dtype=torch.cdouble) for i in range(num_users)}
    data.append(row)

H = pd.DataFrame(data)

In [ ]:
V_col = []
V_init_col = []

for idx, row in H.iterrows():
    H_dict = {str(i): row[i] for i in range(len(row))}
    wmm = WMMSE_alg_sc(K=num_users, n_tx=n_tx, n_rx=n_rx, H=H_dict, PT=PT, sig_k=sig, d=d, alpha=alpha, max_iter_alg=max_iter_alg, tol_alg=tol_alg)
    V_init = init_V(H_dict)
    V_init_proj = proj_power(V_init, PT)
    V_l, U_l, W_l = wmm.algorithm(V_init_proj)
    V_init_col.append(V_init_proj)
    V_col.append(V_l[-1])

V_df = pd.DataFrame(V_col)
V_init_df = pd.DataFrame(V_init_col)

dset = pd.concat([H, V_init_df, V_df], axis=1)

In [ ]:
s_rate_l = []
for idx, row in H.iterrows():
    s_rate = 0
    H_dict = {str(i): row[i] for i in range(len(row))}
    wmm = WMMSE_alg_sc(K=num_users, n_tx=n_tx, n_rx=n_rx, H=H_dict, PT=PT, sig_k=sig, d=d, alpha=alpha, max_iter_alg=max_iter_alg, tol_alg=tol_alg)
    V_init = init_V(H_dict)
    V_init_proj = proj_power(V_init, PT)
    V_l, U_l, W_l = wmm.algorithm(V_init_proj)
    for i in range(num_users):
        s_rate += torch.log2(torch.linalg.det(W_l[-1][str(i)]))
    s_rate_l.append(s_rate)

In [ ]:
set_up = setup(n_tx=n_tx, n_rx=n_rx, num_streams=d, num_users=num_users, PT=PT, sig = sig)

du = DUNN(num_layers=2, setup=set_up)

tr = Trainer(model=du, setup=set_up, lr=1e-1)
V_final = tr.train_epoch(dset=dset, loss_fn=sum_rate_loss_BC, num_epochs=500, batch_size=1)


In [ ]:
m1 = du.layers[0].X_U['0']
m2 = du.layers[0].Y_U['0']
m3 = du.layers[0].Z_U['0']
m4 = du.layers[0].O_U['0']
m5 = du.layers[0].X_W['0']
m6 = du.layers[0].Y_W['0']
m7 = du.layers[0].Z_W['0']
m8 = du.layers[0].X_V['0']
m9 = du.layers[0].Y_V['0']
m10 = du.layers[0].Z_V['0']
m11 = du.layers[0].O_V['0']

In [ ]:
V_col = []
V_init_col = []

for idx, row in H.iterrows():
    H_dict = {str(i): row[i] for i in range(len(row))}
    wmm = WMMSE_alg_sc(K=num_users, n_tx=n_tx, n_rx=n_rx, H=H_dict, PT=PT, sig_k=sig, d=d, alpha=alpha, max_iter_alg=max_iter_alg, tol_alg=tol_alg)
    V_init = init_V(H_dict)
    V_init_proj = proj_power(V_init, PT)
    V_l, U_l, W_l = wmm.algorithm(V_init_proj)
    # V_init_col.append(V_init_proj)
    V_init_col.append(V_final[str(0)])
    V_col.append(V_l[-1])

V_df = pd.DataFrame(V_col)
V_init_df = pd.DataFrame(V_init_col)

dset = pd.concat([H, V_init_df, V_df], axis=1)

In [ ]:
set_up = setup(n_tx=n_tx, n_rx=n_rx, num_streams=d, num_users=num_users, PT=PT, sig = sig)

du = DUNN(num_layers=1, setup=set_up)

tr = Trainer(model=du, setup=set_up, lr=1e-1)
V_final = tr.train_epoch(dset=dset, loss_fn=sum_rate_loss_BC, num_epochs=500, batch_size=1)


In [ ]:
m01 = du.layers[0].X_U['0']
m02 = du.layers[0].Y_U['0']
m03 = du.layers[0].Z_U['0']
m04 = du.layers[0].O_U['0']
m05 = du.layers[0].X_W['0']
m06 = du.layers[0].Y_W['0']
m07 = du.layers[0].Z_W['0']
m08 = du.layers[0].X_V['0']
m09 = du.layers[0].Y_V['0']
m010 = du.layers[0].Z_V['0']
m011 = du.layers[0].O_V['0']

In [ ]:
set_up = setup(n_tx=n_tx, n_rx=n_rx, num_streams=d, num_users=num_users, PT=PT, sig = sig)

du = DUNN(num_layers=2, setup=set_up)

tr = Trainer(model=du, setup=set_up, lr=1e-1)
V_final = tr.train_epoch(dset=dset, loss_fn=sum_rate_loss_BC, num_epochs=500, batch_size=1)
